# Qwen Edit Multiple Angles – ComfyUI Colab

Clean, pinned ComfyUI setup for Qwen Image Edit (Multiple Angles) with automated batch processing.

## Step 1: Environment & ComfyUI Setup

In [ ]:

from pathlib import Path
import os

USE_GOOGLE_DRIVE = True
UPDATE_COMFY_UI = True
USE_COMFYUI_MANAGER = True
INSTALL_CUSTOM_NODES_DEPENDENCIES = True

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

WORKSPACE = "/content/ComfyUI"
if not os.path.exists(WORKSPACE):
    !git clone https://github.com/comfyanonymous/ComfyUI $WORKSPACE

%cd /content/ComfyUI
if UPDATE_COMFY_UI:
    !git pull


## Step 2: Install Fixed Dependencies (No Hub Conflicts)

In [ ]:

!pip install --upgrade pip -q
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q
!pip install accelerate einops safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3 sentencepiece soundfile kornia>=0.7.1 spandrel av sageattention -q
!pip install "transformers>=4.45.0,<4.57.0" "huggingface-hub>=0.23.0,<1.0" -q
!pip install hf_transfer -q


## Step 3: ComfyUI Manager & Custom Nodes

In [ ]:

if USE_COMFYUI_MANAGER:
    %cd custom_nodes
    if not os.path.exists("ComfyUI-Manager"):
        !git clone https://github.com/ltdrdata/ComfyUI-Manager
    %cd ComfyUI-Manager
    !git pull
    %cd /content/ComfyUI

if INSTALL_CUSTOM_NODES_DEPENDENCIES:
    !pip install GitPython -q
    !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
    !pip install comfyui-workflow-templates comfyui-embedded-docs --upgrade --quiet


## Step 4: Hard Fixes & Forced Pins

In [ ]:

!pip install av comfy_aimdo torchsde comfy-kitchen --quiet

!pip uninstall -y huggingface-hub transformers tokenizers -q
!pip install "huggingface-hub==0.36.0" "transformers==4.55.2" "tokenizers==0.21.0" --no-deps -q
!pip install hf_transfer -q

import importlib.metadata as metadata
for p in ["huggingface-hub","transformers","tokenizers"]:
    print(p, metadata.version(p))


## Step 5: Batch Image Automation

In [ ]:

INPUT_FOLDER = "/content/drive/MyDrive/multi-angle/input"
PROCESSED_FOLDER = "/content/drive/MyDrive/multi-angle/processed"
OUTPUT_FOLDER = "/content/drive/MyDrive/multi-angle/output"

WORKFLOW_PATH = "/content/drive/MyDrive/workflows/qwen_3_angles_only_clean_api.json"
IMAGE_INPUT_NODE = "25"

COMFYUI_PORT = 8188
FOLDER_CHECK_INTERVAL = 15
COOLDOWN_BETWEEN_JOBS = 5
MAX_WAIT_TIME = 600


In [ ]:

import os, time, json, requests, subprocess, shutil, threading, socket, glob
from datetime import datetime
from google.colab import output
from pathlib import Path

for d in [INPUT_FOLDER, PROCESSED_FOLDER, OUTPUT_FOLDER]:
    os.makedirs(d, exist_ok=True)

COMFYUI_ROOT = "/content/ComfyUI"
COMFYUI_INPUT_DIR = f"{COMFYUI_ROOT}/input"
COMFYUI_OUTPUT_DIRS = [f"{COMFYUI_ROOT}/output", f"{COMFYUI_ROOT}/temp"]

COMFYUI_URL = f"http://127.0.0.1:{COMFYUI_PORT}"
PROMPT_URL = f"{COMFYUI_URL}/prompt"
HISTORY_URL = f"{COMFYUI_URL}/history"


In [ ]:

def start_comfyui():
    p = subprocess.Popen(
        ["python", "main.py", "--port", str(COMFYUI_PORT), "--dont-print-server"],
        cwd=COMFYUI_ROOT,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    def stream():
        for l in iter(p.stdout.readline, ''):
            print(l, end='')
    threading.Thread(target=stream, daemon=True).start()

    def iframe():
        while True:
            time.sleep(1)
            s = socket.socket()
            if s.connect_ex(("127.0.0.1", COMFYUI_PORT)) == 0:
                break
        output.serve_kernel_port_as_window(COMFYUI_PORT)
    threading.Thread(target=iframe, daemon=True).start()

start_comfyui()


## Step 6: Processing Loop

In [ ]:

with open(WORKFLOW_PATH) as f:
    base_workflow = json.load(f)

def get_images():
    imgs = []
    for p in ['*.png','*.jpg','*.jpeg','*.webp']:
        imgs += glob.glob(f"{INPUT_FOLDER}/{p}")
    return sorted(imgs)

def queue(image):
    wf = json.loads(json.dumps(base_workflow))
    wf[IMAGE_INPUT_NODE]["inputs"]["image"] = image
    r = requests.post(PROMPT_URL, json={"prompt": wf})
    return r.json()["prompt_id"]

print("Drop images into input folder to start processing")

while True:
    imgs = get_images()
    if not imgs:
        time.sleep(FOLDER_CHECK_INTERVAL)
        continue
    for img in imgs:
        name = os.path.basename(img)
        shutil.copy(img, f"{COMFYUI_INPUT_DIR}/{name}")
        pid = queue(name)
        time.sleep(COOLDOWN_BETWEEN_JOBS)
